# Analyse des variables catégorielles

#### Comprendre le contenu des colonnes non numériques et préparer leur utilisation dans un modèle.

In [2]:
import pandas as pd
df = pd.read_csv('../6.Data/kaggle_b2_fraud_train_v3.csv')

In [8]:
categorical_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
summary_df = pd.DataFrame({
    'colonne': categorical_cols,
    'n_valeurs_uniques': [df[col].nunique() for col in categorical_cols]
})
summary_df.sort_values(by='n_valeurs_uniques', ascending=False)


categorical_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
int_categorical_cols = [
    'post_event_status_code',
    'referrer_code',
    'postal_code',
    'manual_review_result',
    'is_vpn',
    'is_new_device',
    'terms_accepted_flag',
    'partner_risk_indicator'  
]
all_categorical_cols = list(set(categorical_cols + int_categorical_cols))

summary_df = pd.DataFrame({
    'colonne': all_categorical_cols,
    'n_valeurs_uniques': [df[col].nunique() for col in all_categorical_cols]
})
summary_df.sort_values(by='n_valeurs_uniques', ascending=False)



C:\Users\yannc\AppData\Local\Temp\ipykernel_13048\2262560441.py:1: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
C:\Users\yannc\AppData\Local\Temp\ipykernel_13048\2262560441.py:9: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See http

,colonne,n_valeurs_uniques
17,customer_id,157836
16,account_id,157836
15,referrer_code,157836
6,postal_code,77910
7,secondary_email,12449
22,partner_risk_indicator,4691
25,signup_date,730
8,city,81
20,region,16
9,country,13


| Colonne                | Explication concise                                   | Nb valeurs uniques | Numériser ? / Commentaire                    |
| ---------------------- | ----------------------------------------------------- | ------------------ | -------------------------------------------- |
| customer_id            | Identifiant unique du client                          | 157836             | Non                       |
| account_id             | Identifiant unique du compte client                   | 157836             | Non                       |
| referrer_code          | Code marketing ou de parrainage                       | 157836             | Non   |
| postal_code            | Code postal du client                                 | 77910              | A voir, faire des zones ?  |
| secondary_email        | Email secondaire du client                            | 12449              | Non     |
| partner_risk_indicator | Indicateur de risque fourni par un partenaire externe | 4691               | Oui, one-hot ou label encoding               |
| signup_date            | Date de création du compte                            | 730                | Oui, extraire features (jour, mois, année)   |
| city                   | Ville du client                                       | 81                 | Oui, one-hot ou regrouper les petites villes |
| region                 | Région administrative                                 | 16                 | Oui, one-hot ou label encoding               |
| country                | Pays du client                                        | 13                 | Oui, one-hot ou label encoding               |
| occupation             | Statut professionnel déclaré                          | 11                 | Oui, one-hot ou label encoding               |
| last_ticket_subject    | Sujet du dernier ticket de support                    | 10                 | Non             |
| customer_note          | Note interne concernant le client                     | 10                 | Non             |
| payment_method         | Méthode de paiement principale                        | 10                 | Oui, one-hot ou label encoding               |
| merchant_category      | Catégorie de marchand la plus utilisée                | 9                  | Oui, one-hot ou label encoding               |
| browser                | Navigateur utilisé par le client                      | 6                  | Oui, one-hot                                 |
| os                     | Système d’exploitation utilisé                        | 6                  | Oui, one-hot                                 |
| signup_source          | Source d’acquisition du client                        | 6                  | Oui, one-hot ou label encoding               |
| device_type            | Type d’appareil utilisé (phone, desktop, tablet)      | 5                  | Oui, one-hot                                 |
| post_event_status_code | Code interne d’événement                              | 5                  | Oui, one-hot ou label encoding               |
| plan_type              | Type de plan ou abonnement                            | 4                  | Oui, one-hot ou label encoding               |
| channel                | Canal principal d’interaction (web, app…)             | 4                  | Oui, one-hot ou label encoding               |
| manual_review_result   | Résultat d’un examen manuel interne                   | 3                  | Oui, label encoding (3 classes)              |
| is_vpn                 | Connexion via VPN ou non                              | 2                  | Oui, 0/1                                     |
| is_new_device          | Nouveau device détecté récemment                      | 2                  | Oui, 0/1                                     |
| terms_accepted_flag    | Acceptation des conditions                            | 1                  | Non, valeur constante, inutile pour ML       |


1️⃣ One-Hot Encoding

Principe :

Chaque catégorie d’une colonne devient une nouvelle colonne binaire (0 ou 1).

Si une ligne a cette catégorie, on met 1, sinon 0

Nouvelle colone (si binaire 2) Avec 1 si on était dedans et 0 sinon

2️⃣ Label Encoding

Principe :

Chaque catégorie devient un entier unique.

Le modèle voit la catégorie comme un nombre, mais attention : certains modèles peuvent interpréter cet ordre comme une hiérarchie, ce qui peut être faux.

| plan_type |  plan_type_encoded |
| --------- |----------------- |
| basic     |0 |
| pro       |2|
| premium   |1|
| basic     |0|



Target Encoding (Encodage par la cible)

Principe :

Pour chaque catégorie d’une variable, on calcule la statistique de la variable cible (target) correspondante.

En général, on prend la moyenne de la target pour chaque catégorie.

Chaque catégorie est ensuite remplacée par ce nombre.

Exemple :

Variable : plan_type
Target : target_is_fraud (0 ou 1)

| plan_type | target_is_fraud |
| --------- | --------------- |
| basic     | 0               |
| pro       | 1               |
| premium   | 0               |
| basic     | 1               |
| pro       | 1               |


Étape 1 : calculer la moyenne de la target par catégorie

| plan_type | target_mean   |
| --------- | ------------- |
| basic     | (0+1)/2 = 0.5 |
| pro       | (1+1)/2 = 1.0 |
| premium   | 0             |


Étape 2 : remplacer les catégories par la moyenne

| plan_type_encoded |
| ----------------- |
| 0.5               |
| 1.0               |
| 0                 |
| 0.5               |
| 1.0               |
